# Libraries


In [20]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from hazm import Normalizer, word_tokenize


## Comments Data Extraction

In [21]:
comments = pd.read_csv("comments.csv")


In [22]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100008 entries, 0 to 100007
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   product_id    100008 non-null  int64 
 1   confirmed_at  100008 non-null  object
 2   comment       99903 non-null   object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [23]:
comments.describe()


,product_id
count,100008.000000
mean,333626.644818
std,163423.773178
min,281.000000
25%,198318.500000
50%,342300.000000
75%,474238.500000
max,684859.000000


In [24]:
comments.isna().sum()


product_id        0
confirmed_at      0
comment         105
dtype: int64

## Sentiments Analysis


In [25]:
comments = comments[["product_id", "comment"]]
comments

,product_id,comment
0,90213,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1,56871,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...
2,49738,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...
3,134367,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...
4,151402,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...
...,...,...
100003,104998,قیمت مناسبی داره در حال حاضر . با این پول سخت ...
100004,333389,در کل راضیم درسته لنوو با قیمت پایین تر هسته و...
100005,356546,من این محصول رو از بیرون تهیه کردم . و باید بگ...
100006,33874,دوسال و.نیم پیش از دیجی کالا گرفتم. هنوز مثل ر...


### Data Cleaning


In [26]:
comments.dropna(subset=["comment"], inplace=True)
comments.isna().sum()

product_id    0
comment       0
dtype: int64

In [27]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99903 entries, 0 to 100007
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  99903 non-null  int64 
 1   comment     99903 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


### Loading StopWords for ENG , PER

In [28]:

with open('stop-words.txt', 'r', encoding='utf-8') as f:
    persian_stopwords = f.read().splitlines()


with open('english', 'r', encoding='utf-8') as f:
    english_stopwords = f.read().splitlines()

print(english_stopwords)
print(persian_stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

### Arranging two StopWords in a DataFrame

In [29]:
all_stopwords = list(set(persian_stopwords + english_stopwords))
all_stopwords = pd.DataFrame(all_stopwords)
all_stopwords

,0
0,كاشكی
1,یا
2,اشد
3,سهوا
4,بی تفاوتند
...,...
2602,نشون
2603,كنایه
2604,اقلا
2605,عری


### First Approach : Regex Applying

In [30]:
def clean_text(text):
    text = str(text)
    # حذف کاراکترهای غیر از حروف فارسی، انگلیسی و اعداد و فاصله
    text = re.sub(r'[^\u0600-\u06FF\uFB50-\uFDFFa-zA-Z0-9\s]', ' ', text)
    # نرمال‌سازی فاصله
    text = re.sub(r'\s+', ' ', text).strip()
    return text
df = comments
df['cleaned_text'] = df['comment'].apply(clean_text)
df['cleaned_text']

0         سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
1         بوی تند ولی خوشبو داره ماندگاریش خوبه و هر چقد...
2         متاسفانه عمر مفید این ماشین کم هست و بعد از دو...
3         افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...
4         این تیغ محصول کشور آمریکا هست در عکس محصول مشخ...
                                ...                        
100003    قیمت مناسبی داره در حال حاضر با این پول سخت می...
100004    در کل راضیم درسته لنوو با قیمت پایین تر هسته و...
100005    من این محصول رو از بیرون تهیه کردم و باید بگم ...
100006    دوسال و نیم پیش از دیجی کالا گرفتم هنوز مثل رو...
100007    با استفاده از این شامپو مشکل خشکی مو و پوست سر...
Name: cleaned_text, Length: 99903, dtype: object

#### کلمات با بیشترین تکرار


In [31]:
word_counts = Counter(df['cleaned_text'])
top_50_words = word_counts.most_common(50)
top_50_words

[('عالی', 93),
 ('بسیار عالی', 76),
 ('حتما پیشنهاد میکنم', 55),
 ('خیلی عالیه', 50),
 ('واقعا عالیه', 44),
 ('عالیه', 37),
 ('در کل خوبه', 35),
 ('پیشنهاد میکنم', 34),
 ('نسبت به قیمتش خوبه', 32),
 ('عالیه حرف نداره', 29),
 ('در یک کلام عالی', 28),
 ('حتما بخرید', 27),
 ('اصلا خوب نیست', 27),
 ('عالی عالی عالی', 27),
 ('نسبت به قیمتش عالیه', 25),
 ('ارزش خرید نداره', 23),
 ('عالیه حتما بخرید', 23),
 ('پیشنهاد نمیکنم', 22),
 ('ارزش خرید داره', 22),
 ('اصلا پیشنهاد نمیکنم', 20),
 ('خیلی خوبه', 20),
 ('اصلا نخرید', 19),
 ('حتما پیشنهاد میکنم ممنون از دیجی', 17),
 ('عالیه حتما پیشنهاد میکنم', 17),
 ('عالیه عالی', 16),
 ('خیلی خوبه حتما بخرید', 16),
 ('در یک کلام عالیه', 16),
 ('فوق العاده', 15),
 ('از همه نظر عالیه', 15),
 ('عالی حتما بخرید', 14),
 ('از هر نظر عالیه', 14),
 ('اصلا بدرد نمیخوره', 13),
 ('در خریدش شک نکنید', 13),
 ('فقط میتونم بگم عالیه', 13),
 ('بسیار زیبا', 13),
 ('واقعا عالی', 13),
 ('بسیار بی کیفیت', 13),
 ('بدرد نمیخوره', 12),
 ('کیفیت پایین', 12),
 ('ممنون از دیجی کال

In [32]:
# text_column = 'comment'
# text_column

### Second Approach :  Hazm Applying

In [33]:
normalizer = Normalizer()
def hazm_preprocessing(text):
    text = str(text)

    normalized_text = normalizer.normalize(text)

    tokens = word_tokenize(normalized_text)

    filtered_tokens = [
        word for word in tokens
        if word not in all_stopwords and (len(word) > 1 or word.isdigit())
    ]
    return " ".join(filtered_tokens)

In [34]:
df['processed_text'] = df['comment' ].apply(hazm_preprocessing)
df['processed_text']

0         سلام قبل اینکه نظرم رو بگم می‌خواستم به یک موض...
1         بوی تند ولی خوشبو داره ماندگاریش خوبه هر چقدر ...
2         متاسفانه عمر مفید این ماشین کم هست بعد از دو س...
3         افا از وقتی من اینو واسه پسرم خریدم همه بچه‌ها...
4         این تیغ محصول کشور آمریکا هست در عکس محصول مشخ...
                                ...                        
100003    قیمت مناسبی داره در حال حاضر با این پول سخت می...
100004    در کل راضیم درسته لنوو با قیمت پایین‌تر هسته و...
100005    من این محصول رو از بیرون تهیه کردم باید بگم زی...
100006    دوسال نیم پیش از دیجی کالا گرفتم هنوز مثل روز ...
100007    با استفاده از این شامپو مشکل خشکی مو پوست سر ض...
Name: processed_text, Length: 99903, dtype: object

In [35]:
# حذف ردیف‌هایی که پس از پردازش کاملاً خالی شده‌اند
df_filtered = df[df['processed_text'].astype(str).str.strip() != ""].copy()
df_filtered.to_csv('Edited_text_comments.csv', index=False)
df_filtered

,product_id,comment,cleaned_text,processed_text
0,90213,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,سلام قبل اینکه نظرم رو بگم می‌خواستم به یک موض...
1,56871,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,بوی تند ولی خوشبو داره ماندگاریش خوبه و هر چقد...,بوی تند ولی خوشبو داره ماندگاریش خوبه هر چقدر ...
2,49738,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,متاسفانه عمر مفید این ماشین کم هست و بعد از دو...,متاسفانه عمر مفید این ماشین کم هست بعد از دو س...
3,134367,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...,افا از وقتی من اینو واسه پسرم خریدم همه بچه ها...,افا از وقتی من اینو واسه پسرم خریدم همه بچه‌ها...
4,151402,این تیغ محصول کشور آمریکا هست ( در عکس محصول م...,این تیغ محصول کشور آمریکا هست در عکس محصول مشخ...,این تیغ محصول کشور آمریکا هست در عکس محصول مشخ...
...,...,...,...,...
100003,104998,قیمت مناسبی داره در حال حاضر . با این پول سخت ...,قیمت مناسبی داره در حال حاضر با این پول سخت می...,قیمت مناسبی داره در حال حاضر با این پول سخت می...
100004,333389,در کل راضیم درسته لنوو با قیمت پایین تر هسته و...,در کل راضیم درسته لنوو با قیمت پایین تر هسته و...,در کل راضیم درسته لنوو با قیمت پایین‌تر هسته و...
100005,356546,من این محصول رو از بیرون تهیه کردم . و باید بگ...,من این محصول رو از بیرون تهیه کردم و باید بگم ...,من این محصول رو از بیرون تهیه کردم باید بگم زی...
100006,33874,دوسال و.نیم پیش از دیجی کالا گرفتم. هنوز مثل ر...,دوسال و نیم پیش از دیجی کالا گرفتم هنوز مثل رو...,دوسال نیم پیش از دیجی کالا گرفتم هنوز مثل روز ...


####  ---  TF-IDF نمایش 50 کلمه برتر ---

In [36]:

"""
(?u)\b\w+\b
Regex Operation for recognition of suitable characters
"""


if not df_filtered.empty:
    # TF-IDF با پارامترهای مناسب برای متن توکنایز شده
    tfidf_vectorizer = TfidfVectorizer(
        token_pattern=r'(?u)\b\w+\b'
    )

    # محاسبه ماتریس TF-IDF بر روی متن‌های پردازش شده با hazm
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_filtered['processed_text'].astype(str))

    # دریافت نام ویژگی‌ها (کلمات)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # محاسبه مجموع امتیازات TF-IDF برای هر کلمه در کل مجموعه
    sums = tfidf_matrix.sum(axis=0)
    data = []
    for col, term in enumerate(feature_names):
        data.append((term, sums[0, col]))

    ranking_df = pd.DataFrame(data, columns=['word', 'tfidf_score_sum'])

    # مرتب‌سازی بر اساس امتیاز TF-IDF و نمایش 50 کلمه برتر
    top_50_tfidf = ranking_df.sort_values(by='tfidf_score_sum', ascending=False).head(50)

top_50_tfidf

,word,tfidf_score_sum
7586,از,3780.799040
67898,که,3610.680728
27318,خیلی,3565.039360
18197,به,3528.224968
12867,این,3194.912195
48840,من,2976.779339
50255,می,2756.717121
13271,با,2676.178804
56500,هم,2618.545708
32350,رو,2516.181019


####  کلمات با بیشترین تکرار
50 کلمه

In [37]:
df['comment'].isna().sum()
df_text_column = df['comment'].dropna()

In [38]:
word_counts = Counter(df_text_column)
top_50_words = word_counts.most_common(50)
top_50_words

[('عالی', 84),
 ('بسیار عالی', 74),
 ('حتما پیشنهاد میکنم', 55),
 ('خیلی عالیه', 48),
 ('واقعا عالیه', 41),
 ('عالیه', 34),
 ('در کل خوبه', 33),
 ('پیشنهاد میکنم', 32),
 ('نسبت به قیمتش خوبه', 29),
 ('اصلا خوب نیست', 26),
 ('در یک کلام عالی', 26),
 ('حتما بخرید', 25),
 ('عالی عالی عالی', 25),
 ('عالیه حرف نداره', 24),
 ('نسبت به قیمتش عالیه', 24),
 ('ارزش خرید داره', 22),
 ('ارزش خرید نداره', 21),
 ('عالیه حتما بخرید', 21),
 ('پیشنهاد نمیکنم', 21),
 ('اصلا نخرید', 17),
 ('اصلا پیشنهاد نمیکنم', 15),
 ('عالیه عالی', 15),
 ('فوق العاده', 14),
 ('از همه نظر عالیه', 14),
 ('خیلی خوبه حتما بخرید', 14),
 ('عالیه حتما پیشنهاد میکنم', 13),
 ('از هر نظر عالیه', 13),
 ('خیلی خوبه', 13),
 ('در یک کلام عالیه', 13),
 ('کیفیت پایین', 12),
 ('حتما پیشنهاد میکنم ممنون از دیجی', 12),
 ('عالی حتما بخرید', 12),
 ('در خریدش شک نکنید', 12),
 ('بسیار زیبا', 12),
 ('ممنون از دیجی کالا', 12),
 ('بسیار بی کیفیت', 12),
 ('بدرد نمیخوره', 11),
 ('اصلا بدرد نمیخوره', 11),
 ('خیلی خوب بود', 11),
 ('در کل عالیه', 11)